In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np
import re
import json
import requests

URL = "mts-prism.com"
PORT = 8082

# Please do NOT share this information anywhere, unless you want your team to be cooked.
TEAM_API_CODE = "2b57102b8df20938be39b22b5f0dcf25"
# @cyrus or @myguysai on Discord if you need an API key


def send_get_request(path):
    """
    Sends a HTTP GET request to the server.
    Returns:
        (success?, error or message)
    """
    headers = {"X-API-Code": TEAM_API_CODE}
    response = requests.get(f"http://{URL}:{PORT}/{path}", headers=headers)

    # Check whether there was an error sent from the server.
    # 200 is the HTTP Success status code, so we do not expect any
    # other response code.
    if response.status_code != 200:
        return (
            False,
            f"Error - something went wrong when requesting [CODE: {response.status_code}]: {response.text}",
        )
    return True, response.text


def send_post_request(path, data=None):
    """
    Sends a HTTP POST request to the server.
    Pass in the POST data to data, to send some message.
    Returns:
         (success?, error or message)
    """
    headers = {"X-API-Code": TEAM_API_CODE, "Content-Type": "application/json"}

    # Convert the data from python dictionary to JSON string,
    # which is the expected format to be passed
    data = json.dumps(data)
    response = requests.post(f"http://{URL}:{PORT}/{path}", data=data, headers=headers)

    # Check whether there was an error sent from the server.
    # 200 is the HTTP Success status code, so we do not expect any
    # other response code.
    if response.status_code != 200:
        return (
            False,
            f"Error - something went wrong when requesting [CODE: {response.status_code}]: {response.text}",
        )
    return True, response.text


def get_context():
    """
    Query the challenge server to request for a client to design a portfolio for.
    Returns:
        (success?, error or message)
    """
    return send_get_request("/request")


def get_my_current_information():
    """
    Query your team information.
    Returns:
        (success?, error or message)
    """
    return send_get_request("/info")


def send_portfolio(weighted_stocks):
    """
    Send portfolio stocks to the server for evaluation.
    Returns:
        (success?, error or message)
    """
    data = [
        {"ticker": weighted_stock[0], "quantity": weighted_stock[1]}
        for weighted_stock in weighted_stocks
    ]
    return send_post_request("/submit", data=data)


# def get_dates(start, end):
#     ticker = yf.Ticker("KO")
#     start = pd.to_datetime(start)
#     end = pd.to_datetime(end)

#     data_start = ticker.history(start=start, end=start + pd.Timedelta(days=3))
#     data_end = ticker.history(start=end - pd.Timedelta(days=3), end=end)
#     # If there's no data at all (e.g., wrong ticker), return None
#     if data_start.empty or data_end.empty:
#         print("ahhhh")
#         return None
  
#     start = data_start.index[0]
#     end = data_end.index[-1]
#     return start, end

# def get_prices(tickers, period):
#     data_start = yf.download(tickers, start=period[0], end=period[0] + pd.Timedelta(days=1))
#     data_end = yf.download(tickers, start=period[1], end=period[1] + pd.Timedelta(days=1))
#     # Extract Open and Close prices for each ticker
#     open_prices = data_start["Open"].iloc[0]
#     close_prices = data_end["Close"].iloc[0]
#     # Calculate returns
#     returns = (close_prices - open_prices) / open_prices

#     # Build final DataFrame
#     result = pd.DataFrame({
#         'Initial Price': open_prices,
#         'Return': returns
#     })
#     return result

def get_portfolio(tickers, budget, start, end, data):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    # Create an empty DataFrame for the results
    returns_data = pd.DataFrame(columns=["Ticker", "Starting Value", "Return"])
    price_start = data.loc[start][tickers]
    price_end = data.loc[end][tickers]
    returns = (price_end - price_start)/price_start

    returns_data = pd.DataFrame({
    "Price": price_start,
    "Return": returns
    })
    
    best5 = returns_data.nlargest(5, "Return")
    budget_split = np.array([0.4,0.2,0.1,0.05,0.05]) * budget
    port = np.zeros(5, dtype=np.int64)
    for i in range(4,-1,-1):
        port[i] = budget_split[i] // best5.iloc[i]["Price"]
        if i < 0:
            budget_split[i+1] += budget_split[i] - port[i]*best5.iloc[i]["Price"]
    return [(asset, port[i]) for i,asset in enumerate(best5.index)]

from transformers import pipeline
from datetime import datetime

# Load the model and tokenizer
model_name = "deepset/roberta-base-squad2"
nlp = pipeline("question-answering", model=model_name, tokenizer=model_name)

# Predefined list of sectors to avoid
avoid_sectors = [
    "Technology", "Financial Services", "Consumer Defensive", "Industrials",
    "Healthcare", "Energy", "Consumer Cyclical", "Communication Services"
]

def extract_investment_info(message):
    questions = [
        "What is the start date?",
        "What is the end date?",
        "What is the budget?",
        "What companies should be avoided?"
    ]
    answers = {}
    for question in questions:
        result = nlp(question=question, context=message)
        answers[question] = result['answer']
    
    # Extract and format start and end dates
    start_date_raw = answers.get("What is the start date?", "")
    end_date_raw = answers.get("What is the end date?", "")
    try:
        start_date = datetime.strptime(start_date_raw, "%B %dth, %Y").strftime("%Y-%m-%d")
    except ValueError:
        start_date = None  # Handle invalid date format
    try:
        end_date = datetime.strptime(end_date_raw, "%B %dth, %Y").strftime("%Y-%m-%d")
    except ValueError:
        end_date = None  # Handle invalid date format

    # Extract and filter "avoid companies" to only include sectors in the predefined list
    avoid_companies = answers.get("What companies should be avoided?", "")
    filtered_avoid_investment = [
        sector for sector in avoid_sectors if sector.lower() in avoid_companies.lower()
    ]

    # Extract budget
    budget = answers.get("What is the budget?", "")

    # Return the formatted results
    return {
        "Avoids Investment In": filtered_avoid_investment,
        "Start": start_date,
        "End": end_date,
        "Budget": budget
    }

# Example message
# message = (
#     "Tristan White is 61 years old, has a hobby of rock climbing, and has an "
#     "investment start date of July 17th, 2010 and an investment end date of "
#     "November 30th, 2012. He has a budget of $67903 total. Avoid investing in Technology and Energy."
# )

# # Extract information
# info = extract_investment_info(message)
# print(info)

    
    # return {
    #     "Avoids Investment In": filtered_avoid_investment,
    #     "Start": start_date,
    #     "End": end_date,
    #     "Budget": budget
    # }

# Load the tickers and sectors
tickers_df = pd.read_csv(r'C:\Users\Aneurin\python\homestretch\ticker_sector (1).csv')
# Load the historical data
historical_df = pd.read_csv(r"C:\Users\Aneurin\python\homestretch\historical_databam.csv", index_col=0, parse_dates=True)
success, information = get_my_current_information()
if not success:
    print(f"Error: {information}")
print(f"Team information: ", information)
# Example prompt
success, context = get_context()
if not success:
    print(f"Error: {context}")
print(f"Context provided: ", context)

# Parse the context string as JSON
context_data = json.loads(context)
# Construct a plain-text prompt

investment_info = extract_investment_info(context_data["message"])


# Filter the DataFrame to exclude rows with sectors in "Avoids Investment In" (case-insensitive)
remaining_tickers = tickers_df[~tickers_df['Sector'].str.lower().isin([sector.lower() for sector in investment_info['Avoids Investment In']])]['Ticker']

# Convert the filtered DataFrame column to a pandas Series
remaining_tickers = remaining_tickers.reset_index(drop=True)

portfolio = get_portfolio(remaining_tickers.tolist(), float(investment_info["Budget"]), investment_info["Start"], investment_info["End"], historical_df)
print(portfolio)
success, response = send_portfolio(portfolio)
if not success:
    print(f"Error: {response}")
print(f"Evaluation response: ", response)

# # Save the pandas Series to a CSV file
# remaining_tickers.to_csv(r'C:\Users\owain\Documents\Hackathon\KMITT\mo money\remainingtickers.csv', index=False)


Device set to use cpu


Team information:  {"teamname":"Kiss me thru the isomorphism","points":0,"profit":0,"last_submission_time":"2025-04-05T22:02:10.497751Z"}
Context provided:  {"message":"The investment start date for Latasha is April 4, 2016 and the investment end date is February 16, 2018. She is 65 years old and has a budget of $3317 per year. She avoids Energy and Transportation, Trade and Services. She is employed at True Salary and her budget is $3317."}



ValueError: could not convert string to float: '$3317 per year'